In [1]:
%load_ext autoreload
%autoreload 2

import ast
from pathlib import Path
from pprint import pprint
from docstring_format import *

In [2]:
# file = Path('equation_parser/functions.py')
file = Path('./tests/dummy_tests_functions.py')

raw_text = file.read_text()
dirty_lines = raw_text.splitlines()
tree = ast.parse(raw_text)

classes = [item for item in tree.body if isinstance(item, ast.ClassDef)]

class_methods = [func for item in classes for func in item.body if isinstance(func, ast.FunctionDef)]

functions = [item for item in tree.body if isinstance(item, ast.FunctionDef)]

# File annotations

In [8]:
for func in functions:
    for item in func.args.args:
        print(item.annotation)

None


In [9]:
lines = dirty_lines.copy()
for func in functions:
    lines = annotate_function(func, lines)
    
for method in class_methods:
    lines = annotate_function(method, lines)

In [10]:
dirty_lines

['from typing import Iterable, Optional',
 '',
 '',
 'def function1(arg1: str):',
 '    """',
 '    AZrojrltndflg lejkkjntgdf',
 '',
 '    Parameters',
 '    ----------',
 '    arg1: test',
 '',
 '    Returns',
 '    -------',
 '',
 '    """',
 '    pass',
 '',
 '',
 'def function2(arg1: list, ) -> str:',
 '    """A function doing something',
 '',
 '',
 '    Second paragraph.',
 '    ',
 '    Third paragraph',
 '    Parameters',
 '    ----------',
 '    arg1 (list): This does something',
 '',
 '    """',
 '    pass',
 '',
 '',
 'def function3(arg1: Optional[Iterable] = None) -> dict:',
 '    """',
 '',
 '    Parameters',
 '    ----------',
 '    arg1',
 '',
 '    Returns',
 '    -------',
 '',
 '    """',
 '    pass',
 '',
 '',
 'def function4(arg1: Optional[Iterable] = None, arg2: Optional[tuple] = None) -> dict:',
 '    """',
 '',
 '    Parameters',
 '    ----------',
 '    arg1: this does that',
 '',
 '    Returns',
 '    -------',
 '',
 '    """',
 '    pass',
 '',
 '',
 'def delta

In [11]:
lines

['from typing import Iterable, Optional',
 '',
 '',
 'def function1(arg1: str):',
 '    """',
 '    AZrojrltndflg lejkkjntgdf',
 '',
 '    Parameters',
 '    ----------',
 '    arg1 (str): test',
 '',
 '    Returns',
 '    -------',
 '',
 '    """',
 '    pass',
 '',
 '',
 'def function2(arg1: list, ) -> str:',
 '    """A function doing something',
 '',
 '',
 '    Second paragraph.',
 '    ',
 '    Third paragraph',
 '    Parameters',
 '    ----------',
 '    arg1 (list): This does something',
 '',
 '    """',
 '    pass',
 '',
 '',
 'def function3(arg1: Optional[Iterable] = None) -> dict:',
 '    """',
 '',
 '    Parameters',
 '    ----------',
 '    arg1',
 '',
 '    Returns',
 '    -------',
 '',
 '    """',
 '    pass',
 '',
 '',
 'def function4(arg1: Optional[Iterable] = None, arg2: Optional[tuple] = None) -> dict:',
 '    """',
 '',
 '    Parameters',
 '    ----------',
 '    arg1 (Optional[Iterable]): this does that',
 '',
 '    Returns',
 '    -------',
 '',
 '    """',
 '    p

In [ ]:
start, length = get_docstring_lines(func, dirty_lines)
corrected_docstring = annotate_function(func, dirty_lines)

In [ ]:
def correct_lines(dirty_lines, start, length, corrected_docstring):
    corrected_lines = dirty_lines.copy()
    [corrected_lines.pop(start) for _ in range(length)]
    [corrected_lines.insert(start, line) for line in corrected_docstring.splitlines()[::-1]]
    
    return corrected_lines

In [ ]:
corrected_lines = correct_lines(dirty_lines, start, length, corrected_docstring)

In [ ]:
corrected_lines[start:start+length]

# Unittests

In [ ]:
from docstring_format.base import *
import json

In [ ]:
file = Path('./tests/dummy_tests_functions.py')

raw_text = file.read_text()
dirty_lines = raw_text.splitlines()
tree = ast.parse(raw_text)

classes = [item for item in tree.body if isinstance(item, ast.ClassDef)]
class_methods = [func for item in classes for func in item.body if isinstance(func, ast.FunctionDef)]

functions = [item for item in tree.body if isinstance(item, ast.FunctionDef)]

results = {}
for func in functions:
    start, length = get_docstring_lines(func, dirty_lines)
    results[func.name] = dict(zip(('start', 'length'), (start, length)))
    
    docstring = '\n'.join(dirty_lines[start:start+length])
    results[func.name]['sections'] = get_docstring_sections(docstring).to_dict()
    
    results[func.name]['docstring'] = annotate_function(func, dirty_lines)

In [ ]:
with open('./tests/dummy_results.json', mode='w') as f:
    json.dump(results, f, indent=4)

# Dev 

In [ ]:
from docstring_format.base import *
from docstring_format.constants import *
import json

In [ ]:
file = Path('equation_parser/constants.py')
# file = Path('./tests/dummy_tests_functions.py')

raw_text = file.read_text()
dirty_lines = raw_text.splitlines()
tree = ast.parse(raw_text)

classes = [item for item in tree.body if isinstance(item, ast.ClassDef)]

class_methods = [func for item in classes for func in item.body if isinstance(func, ast.FunctionDef)]

functions = [item for item in tree.body if isinstance(item, ast.FunctionDef)]

In [ ]:
func = functions[0]

docstring = get_docstring(func, dirty_lines)

pprint(docstring.splitlines())

pprint(annotate_function(func, dirty_lines).splitlines())